# Учимся працюваит з БД ORACLE  на Python

## Допоміжна документація
- [Introduction to the Python Driver for Oracle Database](https://python-oracledb.readthedocs.io/en/latest/user_guide/introduction.html#introduction-to-the-python-driver-for-oracle-database)
- [Python python-oracledb Driver](https://oracle.github.io/python-oracledb/)
- [Python: Read Data from Oracle Database](https://kontext.tech/article/1019/python-read-data-from-oracle-database)

## Підключення до БД через "товстий" клієнт ORACLE та виборка одного запису з його виводом через "print"
- Очікується, що вибиирається тільки один запис [5.1.1. Fetch Methods](https://python-oracledb.readthedocs.io/en/latest/user_guide/sql_execution.html#fetch-methods)
- Прив'язка зміних запиту виконується через **змінні прив'язки**

In [3]:
    import oracledb
    import getpass
    
    username = "Oracle Username"
    #userpwd = ""
    userpwd = getpass.getpass("Enter password: ")

    host="10.11.4.16"
    port="1722"
    service_name="TORCL"
    dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'

In [17]:
    import oracledb
    connection = oracledb.connect(dsn)
    with connection.cursor() as cursor:
            sql = """select t.parm_name,t.parm_value from lgw.SOAP_PARM t where parm_name = :pname """
            cursor.execute(sql, {"pname": "DIY.STATUS.URL"})
            row = cursor.fetchone()
            print(row)

    connection.close()


('DIY.STATUS.URL', 'https://pspapi-test.pravex.ua')


## Підключення до БД і втборка кількох ROWS

In [21]:
import oracledb

connection = oracledb.connect(dsn)
cursor = connection.cursor()
cursor.execute("SELECT T.ID_DIVISION, T.PARENT_ID, T.DIVISION, T.DISABLED, T.LONGNAME, T.KTOTD FROM ABS_NSI.ABS$EMPDIV T WHERE ROWNUM<10")
num_rows = 10
while True:
    rows = cursor.fetchmany(size=num_rows)
    if not rows:
        break
    for row in rows:
        print(row)
cursor.close()  
connection.close()

(103619, 103616, 'Відділ генерального секретаріату', 1, 'Отдел генерального секретариата General Secretariat Office', '000')
(103620, 103616, 'Департамент внутрішнього аудиту', 0, 'Департамент внутреннего аудита Internal Audit Department', '000')
(103621, 103620, 'відділ аудиту Головного банку', 1, 'Отдел аудита Головного банка Office of Audit of Head Office', '000')
(103622, 103620, 'Відділ аудиту регіональних центрів та відділень (л)', 1, 'Отдел аудита региональных центров и отделений Audit of Regional Centers and Branches Office', '000')
(103623, 103620, 'Відділ аудиту інформаційних технологій ', 1, 'Отдел аудита информационных технологий IT Audit Office', '000')
(103626, 103625, 'Організаційний відділ', 1, 'Организационный отдел Organization office', '000')
(103627, 103625, 'Відділ управління проектами', 1, 'Отдел управления проектами Project Management office', '000')
(103628, 103616, 'Департамент управління персоналом (л)', 1, 'Департамент управления персоналом Human Resourses Ma

## Визначення метаданим даних: [5.1.3. Query Column Metadata](https://python-oracledb.readthedocs.io/en/latest/user_guide/sql_execution.html#query-column-metadata) та мапінг типів даних: [5.1.4. Fetch Data Types](https://python-oracledb.readthedocs.io/en/latest/user_guide/sql_execution.html#fetch-data-types)

In [10]:
import oracledb

connection = oracledb.connect(dsn)
cursor = connection.cursor()

sql = """SELECT T.* FROM QPF.DIIA$FILE$SGN$ITEMS T WHERE T.IDSF=:A_IDSF"""
cursor.execute(sql, {"A_IDSF": 172236})
for col in cursor.description:
    print(col)
    
num_rows = 10
while True:
    rows = cursor.fetchmany(size=num_rows)
    if not rows:
        break
    for row in rows:
        print(row)
    
connection.close()

('IDSF', <DbType DB_TYPE_NUMBER>, 39, None, 38, 0, False)
('IDSG', <DbType DB_TYPE_NUMBER>, 39, None, 38, 0, True)
('APPFNAME', <DbType DB_TYPE_VARCHAR>, 512, 512, None, None, True)
('APPFCONTENT', <DbType DB_TYPE_BLOB>, None, None, None, None, True)
('APPFHASH', <DbType DB_TYPE_VARCHAR>, 64, 64, None, None, True)
('APPFSIGN', <DbType DB_TYPE_BLOB>, None, None, None, None, True)
('IUSRNM', <DbType DB_TYPE_VARCHAR>, 30, 30, None, None, True)
('IDT', <DbType DB_TYPE_DATE>, 23, None, None, None, True)
('MUSRNM', <DbType DB_TYPE_VARCHAR>, 30, 30, None, None, True)
('MDT', <DbType DB_TYPE_DATE>, 23, None, None, None, True)
(172236, 172235, 'ustav.doc', <oracledb.LOB object at 0x00000216F17224E0>, 'lkOeILR+DTAd0Fe6bHrc/vOhyUHHabHnHY1WN0bZCNs=', None, 'LGW_RMT_MSGB', datetime.datetime(2024, 10, 29, 12, 29, 24), 'LGW_RMT_MSGB', datetime.datetime(2024, 10, 29, 12, 29, 25))


## Пробуємо прочитати дані так, щоб отриманий набір даних трансформувати в Dictionary

In [14]:
import oracledb

connection = oracledb.connect(dsn)
cursor = connection.cursor()

sql = """SELECT T.*FROM QPF.DIIA$FILE$SGN T"""
cursor.execute(sql)

columns = [col[0] for col in cursor.description]
cursor.rowfactory = lambda *args: dict(zip(columns, args))
data = cursor.fetchall()
print(data)
cursor.close()
    
connection.close()


[{'IDSG': 172235, 'IDSYS': 'SEBFL.ONBOARDING    ', 'APPID': 'appdoc-12345656', 'APPUSRNM': None, 'APPDT': None, 'DPL_DLINK': 'https://deeplink/test', 'TRACEID': '1c7a10d7-0143-4476-a970-3611673d287f', 'CRTTRACEID': '2024-10-29T10:29:25.369Z', 'PROCSTATUS': 'SENT', 'PROCSTATUSDSC': None, 'IUSRNM': 'LGW_RMT_MSGB', 'IDT': datetime.datetime(2024, 10, 29, 12, 29, 24), 'MUSRNM': 'LGW_RMT_MSGB', 'MDT': datetime.datetime(2024, 10, 29, 12, 29, 26)}, {'IDSG': 172232, 'IDSYS': 'SEBFL.ONBOARDING    ', 'APPID': 'appdoc-12345678', 'APPUSRNM': None, 'APPDT': None, 'DPL_DLINK': None, 'TRACEID': None, 'CRTTRACEID': None, 'PROCSTATUS': 'NEW', 'PROCSTATUSDSC': None, 'IUSRNM': 'LGW_RMT_MSGB', 'IDT': datetime.datetime(2024, 10, 29, 11, 31, 41), 'MUSRNM': None, 'MDT': None}]


## Вставка даних в БД
- [5.2. INSERT and UPDATE Statements](https://python-oracledb.readthedocs.io/en/latest/user_guide/sql_execution.html#insert-and-update-statements)
  [9. Managing Transactions](https://python-oracledb.readthedocs.io/en/latest/user_guide/txn_management.html#managing-transactions)

In [18]:
import oracledb

connection = oracledb.connect(dsn)
cursor = connection.cursor()

sql = """insert into QPF.DIIA_FILE$SGN_MDF 
          ( IDSYS, APPID, APPUSRNM,  PROCSTATUS) VALUES 
          ( :A_IDSYS, :A_APPID, :A_APPUSRNM, :A_PROCSTATUS )"""

cursor.execute(sql, {"A_IDSYS": "SEBFL.ONBOARDING" , "A_APPID": "PYTEST", "A_APPUSRNM": "USERAPP", "A_PROCSTATUS": "NEW"})

connection.commit()
cursor.close()
    
connection.close()